## ANN for question matching

- Measure Levenshtein distance

- Measure intersection of nouns: normalized 

- Measure the question type: normalized

- Define type of words

In [11]:
import pandas as pd

In [12]:
test=pd.read_csv('test.csv',nrows=200)
train=pd.read_csv('train.csv',nrows=200)
sample_submission=pd.read_csv('sample_submission.csv',nrows=200)

In [10]:
train.question1[0].split(' ')

['What',
 'is',
 'the',
 'step',
 'by',
 'step',
 'guide',
 'to',
 'invest',
 'in',
 'share',
 'market',
 'in',
 'india?']